In [147]:
import pandas as pd
import random
import numpy as np
from random import randint
import time
import torch
from transformers import AutoTokenizer, AutoModel

import time

import memory_profiler

%load_ext memory_profiler

from pathlib import Path

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [148]:
import black
import jupyter_black

jupyter_black.load(line_length=79)

In [149]:
variables_path = Path("../old_variables")
data_path = Path("../data")
# berenslab_data_path = Path("/gpfs01/berens/data/data/GPT_wiki_intro")

# Import

In [5]:
%%time
compression_opts = dict(method="zip", archive_name="iclr.pickle.csv")

iclr = pd.read_pickle(
    data_path / "iclr.pickle.zip",
    # index_col=False,
    compression=compression_opts,
)

CPU times: user 185 ms, sys: 18.1 ms, total: 203 ms
Wall time: 203 ms


In [6]:
iclr

,year,id,title,abstract,authors,decision,scores,keywords,gender-first,gender-last,t-SNE x,t-SNE y
0,2018,ryBnUWb0b,Predicting Floor-Level for 911 Calls with Neur...,"In cities with tall buildings, emergency respo...","William Falcon, Henning Schulzrinne",Accept (Poster),"[7, 6, 6]","[recurrent neural networks, rnn, lstm, mobile ...",male,None,2.536470,0.739367
1,2018,Skk3Jm96W,Some Considerations on Learning to Explore via...,We consider the problem of exploration in meta...,"Bradly Stadie, Ge Yang, Rein Houthooft, Xi Che...",Invite to Workshop Track,"[7, 4, 6]","[reinforcement learning, rl, exploration, meta...",male,male,49.831927,-29.813831
2,2018,r1RQdCg0W,MACH: Embarrassingly parallel $K$-class classi...,We present Merged-Averaged Classifiers via Has...,"Qixuan Huang, Anshumali Shrivastava, Yiqiu Wang",Reject,"[6, 6, 6]","[extreme classification, large-scale learning,...",None,None,-22.502752,9.577367
3,2018,rJ3fy0k0Z,Deterministic Policy Imitation Gradient Algorithm,The goal of imitation learning (IL) is to enab...,"Fumihiro Sasaki, Atsuo Kawaguchi",Reject,"[6, 5, 5]",[imitation learning],None,None,40.437523,-47.690889
4,2018,SkBYYyZRZ,Searching for Activation Functions,The choice of activation functions in deep net...,"Prajit Ramachandran, Barret Zoph, Quoc V. Le",Invite to Workshop Track,"[5, 4, 7]","[meta learning, activation functions]",None,None,-33.260086,-4.038115
...,...,...,...,...,...,...,...,...,...,...,...,...
16531,2023,w4eQcMZsJa,Text-Driven Generative Domain Adaptation with ...,Combined with the generative prior of pre-trai...,"Zhenhuan Liu, Liang Li, Jiayu Xiao, Zhengjun Z...",Desk rejected,[],"[gan, stylegan, clip, domain adaptation, style...",None,None,59.296526,5.206691
16532,2023,SDHSQuBpf2,"Laziness, Barren Plateau, and Noises in Machin...",We define \emph{laziness} to describe a large ...,"Zexi Lin, Liang Jiang",Desk rejected,[],"[theoretical issues in deep learning, learning...",None,male,-29.178083,-21.810583
16533,2023,HyIY8u5LVDr,Discovering the Representation Bottleneck of G...,Most graph neural networks (GNNs) rely on the ...,"Fang Wu, Siyuan Li, Lirong Wu, Dragomir Radev,...",Desk rejected,[],"[gnn bottleneck, graph rewiring, representatio...",None,male,-7.573978,68.386671
16534,2023,470wZ5Qk4ur,Results for Perfect Classification for Graph A...,We study the ability of one layer Graph Attent...,"Kimon Fountoulakis, Amit Levi",Desk rejected,[],[],None,male,-7.753593,60.764583


In [7]:
titles_abstracts_together = [
    iclr.title[i] + " " + iclr.abstract[i] for i in range(len(iclr))
]

In [8]:
print(len(titles_abstracts_together))

16536


# Functions

In [5]:
@torch.no_grad()
def generate_embeddings(abstracts, tokenizer, model, device):
    """Generate embeddings using BERT-based model.
    Code from Luca Schmidt.

    Parameters
    ----------
    abstracts : list
        Abstract texts.
    tokenizer : transformers.models.bert.tokenization_bert_fast.BertTokenizerFast
        Tokenizer.
    model : transformers.models.bert.modeling_bert.BertModel
        BERT-based model.
    device : str, {"cuda", "cpu"}
        "cuda" if torch.cuda.is_available() else "cpu".
        
    Returns
    -------
    embedding_cls : ndarray
        [CLS] tokens of the abstracts.
    embedding_sep : ndarray
        [SEP] tokens of the abstracts.
    embedding_av : ndarray
        Average of tokens of the abstracts.
    """
    # preprocess the input
    inputs = tokenizer(
        abstracts,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=512,
    ).to(device)

    # inference
    outputs = model(**inputs)[0].cpu().detach() 

    embedding_av = torch.mean(outputs, [0, 1]).numpy()
    embedding_sep = outputs[:, -1, :].numpy()
    embedding_cls = outputs[:, 0, :].numpy()

    
    return embedding_cls, embedding_sep, embedding_av 

# Obtain SciBERT embeddings

In [155]:
# random seed
random_state = random.seed(42)

In [156]:
# specify & check gpu usage
device = (
    "cuda" if torch.cuda.is_available() else "cpu"
)  # put cuda:0 if else not working
print("running on device: {}".format(device))

running on device: cuda


In [157]:
# load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
model = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")

print("model: SciBERT uncased")

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model: SciBERT uncased


In [158]:
# set device
model = model.to(device)

## Tokenizer test

In [12]:
type(tokenizer)

transformers.models.bert.tokenization_bert_fast.BertTokenizerFast

In [11]:
type(model)

transformers.models.bert.modeling_bert.BertModel

In [159]:
papers = ["A very very very long and verbose abstract. A very short abstract."]
tokenizer.tokenize(papers[0])

['a',
 'very',
 'very',
 'very',
 'long',
 'and',
 'verb',
 '##ose',
 'abstract',
 '.',
 'a',
 'very',
 'short',
 'abstract',
 '.']

In [27]:
papers = [
    "A very very very long and verbose abstract. A very short abstract",
]
inputs = tokenizer(
    papers,
    padding=True,
    truncation=True,
    return_tensors="pt",
    return_token_type_ids=False,
    max_length=512,
)
print([tokenizer.decode(inputs["input_ids"][i]) for i in range(len(papers))])

['[CLS] a very very very long and verbose abstract. a very short abstract [SEP]']


In [25]:
inputs["input_ids"].shape

torch.Size([1, 6])

## Test to not pass abstracts individually

In [170]:
# define mean pooling function
def sep_pool(model_output, attention_mask):
    token_embeds = model_output[0]
    ix = attention_mask.sum(1) - 1
    ix0 = torch.arange(attention_mask.size(0))
    return token_embeds[ix0, ix, :]

In [151]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[
        0
    ]  # First element of model_output contains all token embeddings
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [161]:
papers = [
    "A very very very long and verbose abstract.",
    "A very short abstract",
]

In [162]:
inputs = tokenizer(
    papers,
    padding=True,
    truncation=True,
    return_tensors="pt",
    return_token_type_ids=False,
    max_length=512,
)
print([tokenizer.decode(inputs["input_ids"][i]) for i in range(len(papers))])

['[CLS] a very very very long and verbose abstract. [SEP]', '[CLS] a very short abstract [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]']


In [163]:
print(len(tokenizer.tokenize(papers[0])))
tokenizer.tokenize(papers[0])

10


['a', 'very', 'very', 'very', 'long', 'and', 'verb', '##ose', 'abstract', '.']

In [164]:
encoded_input = tokenizer(
    papers, padding=True, truncation=True, return_tensors="pt", max_length=512
).to(device)
model_output = model(**encoded_input)

In [179]:
type(model_output)

transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions

In [177]:
output_sep = sep_pool(model_output, encoded_input["attention_mask"])

In [175]:
output_sep.shape

torch.Size([2, 768])

In [ ]:
mean_pooling(model_output, encoded_input['attention_mask'])

In [24]:
token_embeddings = model_output[
    0
]  # First element of model_output contains all token embeddings
attention_mask = encoded_input["attention_mask"]
input_mask_expanded = (
    attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
)

In [58]:
attention_mask.shape

torch.Size([2, 12])

In [52]:
input_mask_expanded.to(torch.bool).shape

torch.Size([2, 12, 768])

In [51]:
input_mask_expanded[1, :, 0]

tensor([1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.], device='cuda:0')

In [26]:
token_embeddings.shape

torch.Size([2, 12, 768])

In [53]:
input_mask_expanded[1, :, :].to(torch.bool).shape

torch.Size([12, 768])

In [66]:
torch.sum(attention_mask, axis=1)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [64]:
token_embeddings[:, torch.sum(attention_mask, axis=1), :].shape

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

## Abstracts + Titles (Update code)

### With batches

In [19]:
%%time

embeddings_cls = []
embeddings_sep = []
embeddings_av = []

batch_size = 1e2
indeces_batches = np.arange(
    0, len(titles_abstracts_together) + batch_size, batch_size, dtype=int
)

for i in range(len(indeces_batches) - 1):
    np.save(variables_path / "experiment_iter", i)

    batch = titles_abstracts_together[
        indeces_batches[i] : indeces_batches[i + 1]
    ]

    embedding_cls, embedding_sep, embedding_av = generate_embeddings(batch, tokenizer, model, device)

    name_embedding_cls = "embedding_batches/embedding_cls_batch_" + str(i)
    name_embedding_sep = "embedding_batches/embedding_sep_batch_" + str(i)
    name_embedding_av = "embedding_batches/embedding_av_batch_" + str(i)

    np.save(
        variables_path / name_embedding_cls,
        embedding_cls,
        allow_pickle=True,
        fix_imports=True,
    )
    np.save(
        variables_path / name_embedding_sep,
        embedding_sep,
        allow_pickle=True,
        fix_imports=True,
    )
    np.save(
        variables_path / name_embedding_av,
        embedding_av,
        allow_pickle=True,
        fix_imports=True,
    )

CPU times: user 8h 18min 2s, sys: 13min, total: 8h 31min 3s
Wall time: 27min 17s


In [20]:
with open(variables_path / "verbose_batches.txt", "w") as f:
    f.write(cap.stdout)

### Join batches

In [21]:
%%time
# join different batches

embeddings_cls = []
embeddings_sep = []
embeddings_av = []


batch_size = 1e2
indeces_batches = np.arange(
    0, len(titles_abstracts_together) + batch_size, batch_size, dtype=int
)

for i in range(len(indeces_batches) - 1):
    name_embedding_cls = (
        "embedding_batches/embedding_cls_batch_" + str(i) + ".npy"
    )
    name_embedding_sep = (
        "embedding_batches/embedding_sep_batch_" + str(i) + ".npy"
    )
    name_embedding_av = (
        "embedding_batches/embedding_av_batch_" + str(i) + ".npy"
    )

    embeddings_cls.append(
        np.load(
            variables_path / name_embedding_cls,
            allow_pickle=True,
            fix_imports=True,
        )
    )
    embeddings_sep.append(
        np.load(
            variables_path / name_embedding_sep,
            allow_pickle=True,
            fix_imports=True,
        )
    )
    embeddings_av.append(
        np.load(
            variables_path / name_embedding_av,
            allow_pickle=True,
            fix_imports=True,
        )
    )


embeddings_cls = np.vstack(embeddings_cls)
embeddings_sep = np.vstack(embeddings_sep)
embeddings_av = np.vstack(embeddings_av)

np.save(
    variables_path / "embeddings_iclr_cls",
    embeddings_cls,
    allow_pickle=True,
    fix_imports=True,
)
np.save(
    variables_path / "embeddings_iclr_sep",
    embeddings_sep,
    allow_pickle=True,
    fix_imports=True,
)
np.save(
    variables_path / "embeddings_iclr_av",
    embeddings_av,
    allow_pickle=True,
    fix_imports=True,
)

CPU times: user 177 ms, sys: 182 ms, total: 359 ms
Wall time: 584 ms


In [22]:
np.max(embeddings_cls - embeddings_av)

12.403848

## Only abstracts (Update code)

In [8]:
abstracts = iclr.abstract.to_list()

In [9]:
print(len(abstracts))

16536


In [10]:
# random seed
random_state = random.seed(42)

In [11]:
# specify & check gpu usage
device = (
    "cuda" if torch.cuda.is_available() else "cpu"
)  # put cuda:0 if else not working
print("running on device: {}".format(device))

running on device: cuda


In [12]:
# load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
model = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")

print("model: SciBERT uncased")

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model: SciBERT uncased


### With batches

In [14]:
%%time

embeddings_cls = []
embeddings_sep = []
embeddings_av = []

batch_size = 1e2
indeces_batches = np.arange(
    0, len(abstracts) + batch_size, batch_size, dtype=int
)

for i in range(len(indeces_batches) - 1):
    np.save(variables_path / "experiment_iter", i)

    batch = abstracts[indeces_batches[i] : indeces_batches[i + 1]]

    embedding_cls, embedding_sep, embedding_av = generate_embeddings(batch, tokenizer, model, device)

    name_embedding_cls = (
        "embedding_batches_abstract/embedding_cls_batch_" + str(i)
    )
    name_embedding_sep = (
        "embedding_batches_abstract/embedding_sep_batch_" + str(i)
    )
    name_embedding_av = "embedding_batches_abstract/embedding_av_batch_" + str(
        i
    )

    np.save(
        variables_path / name_embedding_cls,
        embedding_cls,
        allow_pickle=True,
        fix_imports=True,
    )
    np.save(
        variables_path / name_embedding_sep,
        embedding_sep,
        allow_pickle=True,
        fix_imports=True,
    )
    np.save(
        variables_path / name_embedding_av,
        embedding_av,
        allow_pickle=True,
        fix_imports=True,
    )

tcmalloc: large alloc 1258291200 bytes == 0xb965e000 @ 
tcmalloc: large alloc 1258291200 bytes == 0x7f145858c000 @ 


CPU times: user 8h 6min 34s, sys: 13min 55s, total: 8h 20min 30s
Wall time: 26min 44s


### Join batches

In [16]:
%%time
# join different batches

embeddings_cls = []
embeddings_sep = []
embeddings_av = []


batch_size = 1e2
indeces_batches = np.arange(
    0, len(abstracts) + batch_size, batch_size, dtype=int
)

for i in range(len(indeces_batches) - 1):
    name_embedding_cls = (
        "embedding_batches_abstract/embedding_cls_batch_" + str(i) + ".npy"
    )
    name_embedding_sep = (
        "embedding_batches_abstract/embedding_sep_batch_" + str(i) + ".npy"
    )
    name_embedding_av = (
        "embedding_batches_abstract/embedding_av_batch_" + str(i) + ".npy"
    )

    embeddings_cls.append(
        np.load(
            variables_path / name_embedding_cls,
            allow_pickle=True,
            fix_imports=True,
        )
    )
    embeddings_sep.append(
        np.load(
            variables_path / name_embedding_sep,
            allow_pickle=True,
            fix_imports=True,
        )
    )
    embeddings_av.append(
        np.load(
            variables_path / name_embedding_av,
            allow_pickle=True,
            fix_imports=True,
        )
    )


embeddings_cls = np.vstack(embeddings_cls)
embeddings_sep = np.vstack(embeddings_sep)
embeddings_av = np.vstack(embeddings_av)

np.save(
    variables_path / "embeddings_iclr_abstracts_cls",
    embeddings_cls,
    allow_pickle=True,
    fix_imports=True,
)
np.save(
    variables_path / "embeddings_iclr_abstracts_sep",
    embeddings_sep,
    allow_pickle=True,
    fix_imports=True,
)
np.save(
    variables_path / "embeddings_iclr_abstracts_av",
    embeddings_av,
    allow_pickle=True,
    fix_imports=True,
)

CPU times: user 262 ms, sys: 258 ms, total: 521 ms
Wall time: 971 ms


# Obtain BERT embeddings (Update code)

In [9]:
# random seed
random_state = random.seed(42)

In [10]:
# specify & check gpu usage
device = (
    "cuda" if torch.cuda.is_available() else "cpu"
)  # put cuda:0 if else not working
print("running on device: {}".format(device))

running on device: cuda


In [11]:
# load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

print("model: BERT uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model: BERT uncased


### With batches

In [16]:
%%time

embeddings_cls = []
embeddings_sep = []
embeddings_av = []

batch_size = 1e2
indeces_batches = np.arange(
    0, len(titles_abstracts_together) + batch_size, batch_size, dtype=int
)

for i in range(len(indeces_batches) - 1):
    np.save(variables_path / "experiment_iter", i)

    batch = titles_abstracts_together[
        indeces_batches[i] : indeces_batches[i + 1]
    ]

    embedding_cls, embedding_sep, embedding_av = generate_embeddings(batch, tokenizer, model, device)

    name_embedding_cls = "embedding_batches_bert/embedding_cls_batch_" + str(i)
    name_embedding_sep = "embedding_batches_bert/embedding_sep_batch_" + str(i)
    name_embedding_av = "embedding_batches_bert/embedding_av_batch_" + str(i)

    np.save(
        variables_path / name_embedding_cls,
        embedding_cls,
        allow_pickle=True,
        fix_imports=True,
    )
    np.save(
        variables_path / name_embedding_sep,
        embedding_sep,
        allow_pickle=True,
        fix_imports=True,
    )
    np.save(
        variables_path / name_embedding_av,
        embedding_av,
        allow_pickle=True,
        fix_imports=True,
    )

tcmalloc: large alloc 1209622528 bytes == 0x7f273b870000 @ 
tcmalloc: large alloc 1209622528 bytes == 0x184082000 @ 


CPU times: user 8h 43min 37s, sys: 13min 11s, total: 8h 56min 49s
Wall time: 28min 21s


In [17]:
with open(variables_path / "verbose_batches_bert.txt", "w") as f:
    f.write(cap.stdout)

### Join batches

In [18]:
%%time
# join different batches

embeddings_cls = []
embeddings_sep = []
embeddings_av = []


batch_size = 1e2
indeces_batches = np.arange(
    0, len(titles_abstracts_together) + batch_size, batch_size, dtype=int
)

for i in range(len(indeces_batches) - 1):
    name_embedding_cls = (
        "embedding_batches_bert/embedding_cls_batch_" + str(i) + ".npy"
    )
    name_embedding_sep = (
        "embedding_batches_bert/embedding_sep_batch_" + str(i) + ".npy"
    )
    name_embedding_av = (
        "embedding_batches_bert/embedding_av_batch_" + str(i) + ".npy"
    )

    embeddings_cls.append(
        np.load(
            variables_path / name_embedding_cls,
            allow_pickle=True,
            fix_imports=True,
        )
    )
    embeddings_sep.append(
        np.load(
            variables_path / name_embedding_sep,
            allow_pickle=True,
            fix_imports=True,
        )
    )
    embeddings_av.append(
        np.load(
            variables_path / name_embedding_av,
            allow_pickle=True,
            fix_imports=True,
        )
    )


embeddings_cls = np.vstack(embeddings_cls)
embeddings_sep = np.vstack(embeddings_sep)
embeddings_av = np.vstack(embeddings_av)

np.save(
    variables_path / "embeddings_iclr_bert_cls",
    embeddings_cls,
    allow_pickle=True,
    fix_imports=True,
)
np.save(
    variables_path / "embeddings_iclr_bert_sep",
    embeddings_sep,
    allow_pickle=True,
    fix_imports=True,
)
np.save(
    variables_path / "embeddings_iclr_bert_av",
    embeddings_av,
    allow_pickle=True,
    fix_imports=True,
)

CPU times: user 195 ms, sys: 230 ms, total: 425 ms
Wall time: 784 ms


In [19]:
np.max(embeddings_av - embeddings_cls)

6.90858